# Lab 2 - Model Deplyoment 

In this lab you will deploy a trained model to containers using an Azure Container Instance and and Azure Kubernetes Service using the Azure Machine Learning SDK.

![AML Arch](https://github.com/jakazmie/images-for-hands-on-labs/raw/master/amlarch.png)


In [ ]:
# Verify AML SDK Installed
# view version history at https://pypi.org/project/azureml-sdk/#history 
import azureml.core
print("SDK Version:", azureml.core.VERSION)

## Connect to the workspace

In [ ]:
from azureml.core import Workspace

# Read the workspace config from file
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

## Get the lab datasets

The datasets have been downloaded in Lab 1 into the local folder.


In [ ]:
import os

# Create a temporary folder to store locally relevant content for this notebook
datasetsFolderName = '../datasets'
# os.makedirs(datasetsFolderName, exist_ok=True)
# print('Content files will be saved to {0}'.format(datasetsFolderName))

# filesToDownload = ['UsedCars_Clean.csv', 'UsedCars_Affordability.csv']

# for fileToDownload in filesToDownload:
#  downloadCommand = 'wget -O ''{0}/{1}'' ''https://databricksdemostore.blob.core.windows.net/data/aml-labs/{1}'''.format(datasetsFolderName, fileToDownload)
#  print(downloadCommand)
#  os.system(downloadCommand)
  
# List the downloaded files
os.listdir(datasetsFolderName)

## Train a simple model locally and register it with *Model Registry*

This lab builds upon the lessons learned in the previous lab, but is designed to be self contained. As such we will repeat model training. After training is completed, the trained model will be registered in *Model Registry*.

In [ ]:
# Define a helper method that will train, score and register the classifier using different settings
import numpy as np

from azureml.core.model import Model 
from azureml.core.run import Run
from azureml.core.experiment import Experiment

from sklearn import linear_model 
from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import pickle
import json


def train_eval_register_model(ws, experiment_name, model_name, full_X, full_Y,training_set_percentage):

    # start a training run by defining an experiment
    myexperiment = Experiment(ws, experiment_name)
    run = myexperiment.start_logging()

    train_X, test_X, train_Y, test_Y = train_test_split(full_X, full_Y, train_size=training_set_percentage, random_state=42)

    # Flatten labels
    train_Y = np.ravel(train_Y)
    test_Y = np.ravel(test_Y)
    
    # Convert to float
    train_X = train_X.astype(float)
    test_X = test_X.astype(float)
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(train_X)
    clf = linear_model.LogisticRegression(C=1)
    clf.fit(X_scaled, train_Y)

    scaled_inputs = scaler.transform(test_X)
    predictions = clf.predict(scaled_inputs)
    score = accuracy_score(test_Y, predictions)

    print("With %0.2f percent of data, model accuracy reached %0.4f." % (training_set_percentage, score))

    # Log the training metrics to Azure Machine Learning service run history
    run.log("Training_Set_Percentage", training_set_percentage)
    run.log("Accuracy", score)

    # Serialize the model to a pickle file in the outputs folder
    output_model_path = 'outputs/' + model_name + '.pkl'
    pickle.dump(clf,open(output_model_path,'wb'))
    print('Exported model to ', output_model_path)

    # Serialize the scaler as a pickle file in the same folder as the model
    output_scaler_path = 'outputs/' + 'scaler' + '.pkl'
    pickle.dump(scaler,open(output_scaler_path,'wb'))
    print('Exported scaler to ', output_scaler_path)

    # notice for the model_path, we supply the name of the outputs folder without a trailing slash
    # this will ensure both the model and the scaler get uploaded.
    registered_model = Model.register(model_path='outputs', model_name=model_name, workspace=ws)

    print(registered_model.name, registered_model.id, registered_model.version, sep = '\t')

    run.complete()

    return (registered_model, clf, scaler, score, run)


### Create Experiment and Run training.

In [ ]:
import pandas as pd

# Load our training data set
df_affordability = pd.read_csv('../datasets/UsedCars_Affordability.csv', delimiter=',')

full_X = df_affordability[["Age", "KM"]]
full_Y = df_affordability[["Affordable"]]

# Create an experiment, log metrics and register the created model
experiment_name = "usedcars_training_deployment"
model_name = "usedcarsmodel"
training_set_percentage = 0.50
registered_model, model, scaler, score, run = train_eval_register_model(ws, 
                                                                        experiment_name, 
                                                                        model_name, 
                                                                        full_X, full_Y, 
                                                                        training_set_percentage)


### Download and test version of a model from Azure Machine Learning.

Download the model registered in the previous step and run a quick test.

In [ ]:
# Download the model to a local directory
model_name = 'usedcarsmodel'
scaler_name = 'scaler'

model_path = Model.get_model_path(model_name, _workspace=ws)

# Re-load the model
model = pickle.load(open(os.path.join(model_path, model_name + '.pkl'), 'rb'))
print("Loaded model from:", os.path.join(model_path, model_name + '.pkl'))

# Re-load the scaler
scaler = pickle.load(open(os.path.join(model_path, scaler_name + '.pkl'),'rb'))
print("Loaded scaler from:", os.path.join(model_path, scaler_name + '.pkl'))

# Run a quick test
age = 60
km = 40000

scaled_input = scaler.transform([[age, km]])
prediction = model.predict(scaled_input)
print(prediction)

## Create and deploy the container image encapsulating the model

When you deploy a model using AML to either ACI or AKS, you are deploying a Docker container encapsulating a trained model, its dependencies, and a web services wrapper around the model. 

### Create a Conda dependencies environment file.


In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

mycondaenv = CondaDependencies.create(conda_packages=['scikit-learn','numpy','pandas'])

with open("mydeployenv.yml","w") as f:
    f.write(mycondaenv.serialize_to_string())

Review the content of 'yml' file.

In [ ]:
with open("mydeployenv.yml","r") as f:
    print(f.read())

### Create scoring script.
With Azure Machine Learning, you have full control over the logic of the webservice which includes how it loads your model, transforms web service inputs, uses the model for scoring and returns the result. 

In [ ]:
%%writefile score.py

import json
import os
import numpy as np
import pandas as pd
from sklearn import linear_model 
from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from azureml.core import Run
from azureml.core import Workspace
from azureml.core.run import Run
from azureml.core.experiment import Experiment
import pickle
from sklearn.externals import joblib

def init():
    try:
        # One-time initialization of predictive model and scaler
        from azureml.core.model import Model
        
        global trainedModel   
        global scaler

        model_name = "usedcarsmodel" 
        model_path = Model.get_model_path(model_name)
        print('Looking for models in: ', model_path)

        trainedModel = pickle.load(open(os.path.join(model_path,'usedcarsmodel.pkl'), 'rb'))
        
        scaler = pickle.load(open(os.path.join(model_path,'scaler.pkl'),'rb'))

    except Exception as e:
        print('Exception during init: ', str(e))

def run(input_json):     
    try:
        inputs = json.loads(input_json)

        #Scale the input
        scaled_input = scaler.transform(inputs)
        
        #Get the scored result
        prediction = json.dumps(trainedModel.predict(scaled_input).tolist())

    except Exception as e:
        prediction = str(e)
    return prediction


### Create docker image for deployment

To create a Container Image, you need four things: the model metadata (as retrieved from Model Registry), the scoring script file, the runtime configuration (defining whether Python or PySpark should be used) and the Conda Dependencies file.

In [ ]:
from azureml.core.image import ContainerImage, Image

# Retrieve the model metadata from Model Registry
model_name = 'usedcarsmodel'
model = Model(workspace=ws, name=model_name)

# Define runtime
runtime = "python" 

# Define scoring script
driver_file = "score.py"

# Define conda dependencies
conda_file = "mydeployenv.yml"

# configure the image
image_config = ContainerImage.image_configuration(execution_script=driver_file, 
                                                  runtime=runtime, 
                                                  conda_file=conda_file,
                                                  description="Image for used cars predictor",
                                                  tags={"Classifier": "Logistic regression"})

image = Image.create(name = "used-car-classifier-image",
                     models = [model],
                     image_config = image_config, 
                     workspace = ws)

image.wait_for_creation(show_output = True)


### Deploy the container image to ACI

With the Container Image  in hand, you are almost ready to deploy to ACI. The next step is to define the size of the VM that ACI will use to run your Container.

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice

aci_config = AciWebservice.deploy_configuration(
    cpu_cores = 1, 
    memory_gb = 1, 
    tags = {'name':'Azure ML ACI'}, 
    description = 'This is a deployment of the affordibility predictor.')

At this point you can deploy the image to the webservice to ACI

In [ ]:
from azureml.core.webservice import Webservice

service_name = "usedcarsmlservice-aci"
print("Deploying: ", service_name)
aci_service = Webservice.deploy_from_image(deployment_config = aci_config,
                                           image = image,
                                           name = service_name,
                                           workspace = ws)
aci_service.wait_for_deployment(True)

### Test the service

Once the webservice deployment completes, you can use the returned webservice object to invoke the webservice. 

In [ ]:
import json
age = 60
km = 40000
test_data  = json.dumps([[age,km]])
test_data
result = aci_service.run(input_data=test_data)
print(result)

### Deploy the container image to AKS

Once you are familiar with the process for deploying a webservice to ACI, you will find the process for deploying to AKS to be similar with one additional step that creates the AKS cluster first.

In [ ]:
# Provision an AKS cluster 

from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice

# Use the default configuration, overriding the default location to a known region that supports AKS
prov_config = AksCompute.provisioning_configuration(location='westus2')

aks_name = 'aks-cluster01' 

# Create the cluster
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)


# Wait for cluster to be ready
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

With your AKS cluster ready, now you can deploy your webservice. Once again, you need to provide a configuration for the size of resources allocated from the AKS cluster to run instances of your Container.

In [ ]:
# Create the web service configuration (using defaults)
aks_config = AksWebservice.deploy_configuration()

aks_service_name ='usedcarsmlservice-aks'

aks_service = Webservice.deploy_from_image(
  workspace=ws, 
  name=aks_service_name, 
  image = image,
  deployment_target=aks_target
  )


aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

### Test the service
As before, you can use the webservice object returned by the deploy_from_model method to invoke your deployed webservice. 

In [ ]:
import json
age = 60
km = 40000
test_data  = json.dumps([[age,km]])
test_data
result = aks_service.run(input_data=test_data)
print(result)

## Clean up

Make sure to remove ACI and AKS deployments. Use Azure Portal to remove *Deployments* and *AKS Compute*.